# Creating Custom Types
One of the key features of DEAP is the ability to create custom types to represent individuals and their fitness. In this tutorial, we will explore how to create custom types using the DEAP library.

If you don't have it installed, you can install it using pip:

In [1]:
!pip install deap

In [3]:
#importing the necessary modules from the DEAP library:
import random
from deap import base, creator, tools

## Fitness

In [4]:
# Create the fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [5]:
# Create the individual
creator.create("Individual", list, fitness=creator.FitnessMax)

To put it simply, the first individual created will be a simple list containing floats.
We use the creator.create function to create two custom types:
* FitnessMax: This type represents the fitness of individuals in a maximization problem. We inherit from the base.Fitness class and set the weights attribute to (1.0,) to indicate a single objective problem.
* Individual: This type represents an individual in the population. We inherit from the built-in Python list class and associate it with the fitness attribute (e.g. FitnessMax).



We can also use the type *FitnessMin*

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In DEAP, FitnessMin and FitnessMax are used to represent the fitness values in optimization problems. The choice between FitnessMin and FitnessMax depends on the nature of your optimization problem.

Here's when you should use FitnessMin:

* Minimization Problems: If your problem is a minimization problem, where the goal is to find the minimum value of an objective function, you should use FitnessMin. In this case, the lower the fitness value, the better the individual's fitness.

* Negative Weights: FitnessMin uses negative weights to indicate minimization. By default, the weights attribute of FitnessMin is set to (-1.0,). These negative weights ensure that the fitness values are minimized during the optimization process.

Here's when you should use FitnessMax:

* Maximization Problems: If your problem is a maximization problem, where the goal is to find the maximum value of an objective function, you should use FitnessMax. In this case, the higher the fitness value, the better the individual's fitness.

* Positive Weights: FitnessMax uses positive weights to indicate maximization. By default, the weights attribute of FitnessMax is set to (1.0,). These positive weights ensure that the fitness values are maximized during the optimization process.

It's important to note that the choice between FitnessMin and FitnessMax affects the way fitness values are evaluated and compared during the evolutionary process. It determines the direction of optimization and how selection, crossover, and mutation operators are applied.



The weights attribute in the FitnessMin class determines the minimization or maximization nature of the fitness. It can be customized according to the problem requirements. The weights can also be used to vary the importance of each objective in multi-objective optimization.

In [7]:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))

For instance, if you have a multi-objective optimization problem where you want to minimize the cost (first objective) and maximize the performance (second objective), you can define FitnessMulti with weights=(-1.0, 1.0) to represent these objectives.

## Types of individual

### Permutation

The individual for the permutation representation is similar to the general list individual. The only difference is that instead of filling the list with a series of floats, we generate a random permutation and provide that permutation to the individual.

In [8]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)

c:\Users\hp\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\hp\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In the above code, we create the FitnessMin class for minimizing fitness, and the Individual class as a list type with FitnessMin as its fitness attribute. We register the attribute function indices that generates a random permutation of indices from 0 to 9. Finally, we register the individual function that initializes an individual with the generated permutation.

By generating a random permutation for the individual, we ensure that each index appears exactly once in the individual, representing a valid permutation.

This representation is commonly used in problems where the order or arrangement of elements is important, such as the traveling salesman problem or job scheduling.

### Arithmetic Expression

The next example demonstrates creating an individual for the arithmetic expression representation:

In [10]:
import operator
from deap import creator, gp

pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)

c:\Users\hp\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\hp\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In this example, we create the FitnessMin class for minimizing fitness, and the Individual class as a PrimitiveTree type with FitnessMin as its fitness attribute and the defined primitive set pset. We register the function expr that generates arithmetic expressions using the genHalfAndHalf method. Finally, we register the individual function that initializes an individual with the generated arithmetic expression.

### Evolution Strategy

The next example demonstrates creating an individual for the evolution strategy representation:

In [ ]:
import array

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

def initES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

IND_SIZE = 10
MIN_VALUE, MAX_VALUE = -5.0, 5.0
MIN_STRAT, MAX_STRAT = -1.0, 1.0

toolbox = base.Toolbox()
toolbox.register("individual", initES, creator.Individual,
                 creator.Strategy, IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRAT,
                 MAX_STRAT)


In this example, we create the FitnessMin class for minimizing fitness, the Individual class as an array.array type with FitnessMin as its fitness attribute, and the Strategy class as an array.array type. We define the initES function that initializes an individual for the evolution strategy representation. Finally, we register the individual function that initializes an individual using the initES function.

### Particle
A particle is another special type of individual commonly used in particle swarm optimization. It usually has a speed attribute and remembers its best position. We can create a particle individual by inheriting from the list type. Additionally, we add the speed, best, smin, and smax attributes to the particle class.

In [ ]:
import random

from deap import base, creator, tools

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("particle", initParticle, creator.Particle, size=2,
                 pmin=-6, pmax=6, smin=-3, smax=3)


In this example, we create the FitnessMax class for maximizing fitness, and the Particle class as a list type with FitnessMax as its fitness attribute. We also add the speed, best, smin, and smax attributes to the Particle class. We then define the initParticle() function, which initializes a particle by generating random values within the given ranges for position (pmin to pmax) and speed (smin to smax). This function returns a complete particle individual.

Now, calling toolbox.particle() will create a complete particle individual with a random position and speed, along with a fitness attribute for maximizing two objectives.

### A Funky One
If your problem has specific requirements, it's possible to create custom individuals easily. In this example, we create an individual that consists of a list of alternating integers and floating-point numbers using the initCycle() function.

In [ ]:
import random

from deap import base, creator, tools

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

INT_MIN, INT_MAX = 5, 10
FLT_MIN, FLT_MAX = -0.2, 0.8
N_CYCLES = 4

toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("attr_flt", random.uniform, FLT_MIN, FLT_MAX)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_int, toolbox.attr_flt), n=N_CYCLES)


In this example, we create the FitnessMax class for maximizing fitness, and the Individual class as a list type with FitnessMax as its fitness attribute. We then register two attribute functions, attr_int and attr_flt, which generate random integers and floating-point numbers within the specified ranges. Finally, we register the individual function using tools.initCycle(), which creates an individual by cycling through the provided attribute functions (toolbox.attr_int, toolbox.attr_flt) a specified number of times (n=N_CYCLES).

Now, calling toolbox.individual() will return a complete individual of the form [int, float, int, float, ..., int, float], with a fitness attribute for maximizing two objectives.

## Population
Populations are collections of individuals, strategies, or particles. DEAP provides various population types, such as bag, grid, swarm, and demes.

### Bag
A bag population is the most commonly used type. It has no particular ordering, although it is generally implemented using a list. Since the bag population has no specific attributes, it does not require any special class. We can initialize a bag population directly using the tools.initRepeat() function from the toolbox.

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In this example, we register the population function, which initializes a population using tools.initRepeat() with the list type as the container and toolbox.individual as the initializer. Calling toolbox.population() will return a complete population in the form of a list. You can specify the number of individuals by providing the n argument to the population function. For example, toolbox.population(n=100) will create a population with 100 individuals.



### Grid
A grid population is a special case of a structured population where neighboring individuals have a direct effect on each other. Individuals are distributed in a grid-like structure, where each cell contains a single individual. The grid population is composed of lists of individuals.

In [ ]:
N_COL = 10
N_ROW = 5
toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=N_COL)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=N_ROW)

In this example, we register the row function, which initializes a row of individuals using tools.initRepeat() with the list type as the container and toolbox.individual as the initializer. We specify the number of individuals per row using the n argument. Then, we register the population function, which initializes a population by repeating the row function N_ROW times.

Calling toolbox.population() will return a complete population where individuals are accessible using two indices, for example, pop[r][c]

### Swarm
A swarm is used in particle swarm optimization. It is different from other populations because it contains a communication network among particles. The simplest network is the completely connected one, where each particle knows the best position visited by any particle. This is typically implemented by copying the global best position to a gbest attribute and the global best fitness to a gbestfit attribute.

In [ ]:
creator.create("Swarm", list, gbest=None, gbestfit=creator.FitnessMax)
toolbox.register("particle", initParticle, creator.Swarm, size=2,
                 pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)


### Demes
A deme is a sub-population contained within a larger population. It is similar to an island in the island model. Demes are not fundamentally different from populations, except for their names. We can create a population containing multiple demes, each having a different number of individuals, using the n argument of the tools.initRepeat() function.

In [ ]:
toolbox.register("deme", tools.initRepeat, list, toolbox.individual)

DEME_SIZES = 10, 50, 100
population

### Seeding a Population
We can initialize a population with non-random individuals using the seeding population method.

In [ ]:
# Import the required modules
import json
from deap import base, creator

# Create the fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the initIndividual() function
def initIndividual(icls, content):
    return icls(content)

# Define the initPopulation() function
def initPopulation(pcls, ind_init, filename):
    with open(filename, "r") as pop_file:
        contents = json.load(pop_file)
    return pcls(ind_init(c) for c in contents)

# Create a toolbox instance
toolbox = base.Toolbox()

# Register the individual initializer and population initializer in the toolbox
toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, "my_guess.json")

# Generate the seeded population
population = toolbox.population_guess()


Here we demonstrate how to seed a population using a first guess population from a file. The initIndividual() function is used as the individual initializer, which takes a content as an argument. The initPopulation() function reads the contents from the file "my_guess.json" and creates a population using the individual initializer.

To use this approach, make sure to have a file named "my_guess.json" that contains a list of individuals in the desired format. Then, by executing the code, the population will be initialized using the individuals from the file.

Note that the initIndividual() function and the registration of individual_guess() in the toolbox are optional. If your individual class constructor (creator.Individual) already accepts the content directly, you can skip these steps and directly register the initPopulation() function with the appropriate arguments.



the contents of the "my_guess.json" file can be structured as follows:

In [ ]:
[
  [1, 2, 3, 4, 5],
  [6, 7, 8, 9, 10],
  [11, 12, 13, 14, 15]
]


In this example, the "my_guess.json" file contains a list of three individuals, where each individual is represented as a list of integers. You can customize the contents of the file based on your specific problem and the representation of the individuals.

When you run the code and use the toolbox.population_guess() function, the population will be initialized with the individuals from the "my_guess.json" file.

Make sure to save the "my_guess.json" file in the same directory as your Python script or Jupyter Notebook so that it can be read properly.

# Using Custom Types
Once we have created the custom types, we can utilize them in our evolutionary algorithms. Here's an example of using the custom types:

In [ ]:
# Initialize the toolbox
toolbox = base.Toolbox()

# Register attributes and operators with the toolbox
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Example usage
population = toolbox.population(n=50)
individual = toolbox.individual()
fitness = creator.FitnessMax()


In the above code, we initialize a Toolbox object that serves as a container for various attributes, operators, and algorithms. We register the custom types and their associated functions with the toolbox using the register method. The register() method takes at least two arguments; an alias and a function assigned to this alias.

We can then create individuals and populations using the registered functions. For example, we can initialize a population of 50 individuals using toolbox.population(n=50).

We can also directly create an individual using the toolbox.individual() function, which creates an individual with 10 Boolean attributes.

To create a fitness object of the custom type, we can simply instantiate it using creator.FitnessMax(). This fitness object can be assigned to an individual's fitness attribute.